In [ ]:
import subprocess
import re
import statistics

# List of mail servers to test
mail_servers = {
    "Gmail": "mail.google.com",
    "Outlook": "outlook.office.com",
    "ProtonMail": "mail.proton.me"
}

def ping_server(server):
    """Ping a mail server and return average latency, standard deviation, and packet size."""
    try:
        # Run the Windows ping command (-n 100 sends 100 packets)
        result = subprocess.run(
            ["ping", "-n", "100", server],
            capture_output=True, text=True, check=True
        )
        output = result.stdout

        # Extract all individual ping times: "temps=XXms"
        times = [int(match.group(1)) for match in re.finditer(r"temps[=<]? ?(\d+) ?ms", output)]
        
        # Extract packet size: "octets=32"
        packet_size = None
        match_size = re.search(r"octets=(\d+)", output)
        if match_size:
            packet_size = match_size.group(1)
        
        if times:
            avg_latency = statistics.mean(times)
            std_latency = statistics.stdev(times) if len(times) > 1 else 0  # Avoid error for 1 value
            return avg_latency, std_latency, packet_size
        else:
            return None, None, packet_size
    except subprocess.CalledProcessError:
        return None, None, None

# Run the test
for name, server in mail_servers.items():
    avg, std, packet_size = ping_server(server)
    if avg is not None:
        print(f"{name} ({server}): Moyenne = {avg} ms, Écart-type = {std:.2f} ms, Taille du paquet = {packet_size} octets")
    else:
        print(f"Impossible de ping {name} ({server})")
